In [1]:
%reload_ext autoreload
%autoreload 2

import os
import argparse
import sys
import time

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['GORDON_REPO_DIR'], 'utilities'))
from utilities2015 import *

dm = DataManager(stack='MD594', section=100, segm_params_id='tSLIC200')

In [2]:
centroids = dm.load_pipeline_result('textons')

In [3]:
print 'loading centroids and features ...',
t = time.time()
features_rotated = dm.load_pipeline_result('featuresRotated')
print 'done in', time.time() - t, 'seconds'


loading centroids and features ... done in 24.3020620346 seconds


In [4]:
D = Parallel(n_jobs=16)(delayed(cdist)(fs, centroids) for fs in np.array_split(features_rotated[:10000000], 16))

In [15]:
n_texton = len(centroids)

print 'assign textons ...',
t = time.time()

# from sklearn.cluster import MiniBatchKMeans
from scipy.spatial.distance import cdist

# kmeans = MiniBatchKMeans(n_clusters=n_texton, batch_size=1000, init=centroids, max_iter=1)
# kmeans.fit(features_rotated)
# # final_centroids = kmeans.cluster_centers_
# labels = kmeans.labels_

# D = Parallel(n_jobs=4)(delayed(cdist)(fs, centroids) for fs in np.array_split(features_rotated, 4))

label_list = []
for i in range(0, len(features_rotated), 5000000):
#     D = cdist(fs, centroids)
#     os.system('rm /dev/shm/*')
    Ds = Parallel(n_jobs=16)(delayed(cdist)(fs, centroids) 
                            for fs in np.array_split(features_rotated[i:i+5000000], 16))
    for D in Ds:
        label_list.append(np.argmin(D, axis=1))

labels = np.concatenate(label_list)

print 'done in', time.time() - t, 'seconds'

assign textons ... 0
5000000
10000000
15000000
20000000
25000000
30000000
35000000
40000000
45000000
50000000
55000000
60000000
65000000
70000000
75000000
done in 106.764159918 seconds


In [8]:
textonmap = -1 * np.ones((dm.image_height, dm.image_width), dtype=np.int8)
textonmap[dm.mask] = labels

In [11]:
dm.save_pipeline_result(textonmap, 'texMap')

saved /oasis/projects/nsf/csd395/yuncong/CSHL_data_results/MD594/0100/MD594_0100_lossless_gabor-blueNisslWide-vq-blueNissl_texMap.bp


In [9]:
colors = (np.loadtxt(dm.repo_dir + '/visualization/100colors.txt') * 255).astype(np.uint8)

textonmap_viz = np.zeros((dm.image_height, dm.image_width, 3), np.uint8)
textonmap_viz[dm.mask] = colors[textonmap[dm.mask]]
dm.save_pipeline_result(textonmap_viz, 'texMapViz')

saved /oasis/projects/nsf/csd395/yuncong/CSHL_data_results/MD594/0100/MD594_0100_lossless_gabor-blueNisslWide-vq-blueNissl_texMap.npy
saved /oasis/projects/nsf/csd395/yuncong/CSHL_data_results/MD594/0100/MD594_0100_lossless_gabor-blueNisslWide-vq-blueNissl_texMapViz.jpg


In [ ]:
# reconstruct textons

# from skimage.util import pad
# padded_kernels = [None] * dm.n_kernel
# for i, kern in enumerate(dm.kernels):
#     ksize = kern.shape[0]
#     a = (dm.max_kern_size - ksize)/2
#     padded_kern = pad(kern, [a, a], mode='constant', constant_values=0)
#     padded_kernels[i] = padded_kern
    
    
# F = np.vstack([k.flatten() for k in padded_kernels])

# from numpy.linalg import lstsq

# for f in centroids:
#     b, r, _, _ = lstsq(F, f)

#     c = b.reshape((dm.max_kern_size,dm.max_kern_size))

#     plt.imshow(c[dm.max_kern_size/2-30:dm.max_kern_size/2+30,
#                  dm.max_kern_size/2-30:dm.max_kern_size/2+30], cmap=plt.cm.Greys_r)
    
#     plt.show()